# PostBatchCopyResults
2025-06-11
This notebook moves the results from the individual session output folders to the shared/public CollectedOutputs or AcrossSession folders


In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import os
import sys
import platform
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any

from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files, OldFileArchiver
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers


# # required to enable non-blocking interaction:
# %gui qt5 ## TODO 2024-01-18 - this causes kernel to crash when running notebook remotely via VSCode's ssh remote

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.exception_helpers import CapturedException

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget
from pyphocorehelpers.gui.Jupyter.simple_widgets import code_block_widget

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from pyphocorehelpers.assertion_helpers import Assert

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import build_vscode_workspace, build_windows_powershell_run_script

from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots, BatchPhoJonathanFiguresHelper
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

from neuropy.core.neuron_identities import NeuronIdentityDataframeAccessor
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import build_merged_neuron_firing_rate_indicies
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DirectionalPlacefieldGlobalComputationFunctions, DirectionalLapsHelpers
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, filesystem_path_folder_contents_widget

from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import MAIN_get_template_string, write_test_script
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import export_session_h5_file_completion_function, curr_runtime_context_header_template, export_rank_order_results_completion_function, figures_rank_order_results_completion_function, determine_session_t_delta_completion_function, perform_sweep_decoding_time_bin_sizes_marginals_dfs_completion_function, compute_and_export_decoders_epochs_decoding_and_evaluation_dfs_completion_function, compute_and_export_session_wcorr_shuffles_completion_function, compute_and_export_session_instantaneous_spike_rates_completion_function, compute_and_export_session_extended_placefield_peak_information_completion_function, compute_and_export_session_alternative_replay_wcorr_shuffles_completion_function, backup_previous_session_files_completion_function, compute_and_export_session_trial_by_trial_performance_completion_function, save_custom_session_files_completion_function, compute_and_export_cell_first_spikes_characteristics_completion_function, figures_plot_cell_first_spikes_characteristics_completion_function, generalized_decode_epochs_dict_and_export_results_completion_function
from pyphoplacecellanalysis.General.Batch.BatchJobCompletion.UserCompletionHelpers.batch_user_completion_helpers import kdiba_session_post_fixup_completion_function
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import ProcessingScriptPhases

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category
from pyphoplacecellanalysis.General.Batch.pythonScriptTemplating import BatchScriptsCollection, generate_batch_single_session_scripts, display_generated_scripts_ipywidget


BATCH_DAY_DATE: str = '2025-07-08'
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Apogee' # used for filenames throught the notebook
BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_GL' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_Lab' # used for filenames throught the notebook
# BATCH_DATE_TO_USE = f'{BATCH_DAY_DATE}_rMBP' # used for filenames throught the notebook

known_scripts_output_paths = [Path(v).resolve() for v in ['/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/cloud/turbo/Data/Output/gen_scripts', '/home/halechr/cloud/turbo/Data/Output/gen_scripts/', '/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', '/home/halechr/FastData/gen_scripts/', 'output/gen_scripts/', "K:/scratch/gen_scripts"]]
scripts_output_path = find_first_extant_path(known_scripts_output_paths)
assert scripts_output_path.exists(), f"scripts_output_path: {scripts_output_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'scripts_output_path: {scripts_output_path}')

known_collected_output_paths = [Path(v).resolve() for v in ['/nfs/turbo/umms-kdiba/Data/Output/collected_outputs', '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/data/collected_outputs', 'output/gen_scripts/', 
															'/Volumes/SwapSSD/Data/collected_outputs', '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs', '/home/halechr/FastData/collected_outputs/',
															'/home/halechr/cloud/turbo/Data/Output/collected_outputs', scripts_output_path.joinpath('../collected_outputs').resolve()]]
collected_outputs_path = find_first_extant_path(known_collected_output_paths)
Assert.path_exists(collected_outputs_path) # f"collected_outputs_path: {collected_outputs_path} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
collected_outputs_path.mkdir(exist_ok=True)
print(f'collected_outputs_path: {collected_outputs_path}')

collected_figures_path = collected_outputs_path.parent.joinpath('collected_figures')
collected_figures_path.mkdir(exist_ok=True)
print(f'collected_figures_path: {collected_figures_path}')
Assert.path_exists(collected_figures_path)

# Hardcoded included_session_contexts:
all_session_contexts: List[IdentifyingContext] = UserAnnotationsManager.get_hardcoded_good_sessions()
# included_session_contexts
# excluded_session_keys = ['kdiba_gor01_one_2006-6-08_14-26-15', 'kdiba_gor01_one_2006-6-09_1-22-43',
#     # 'kdiba_gor01_one_2006-6-08_14-26-15', 'kdiba_gor01_two_2006-6-07_16-40-19', 'kdiba_pin01_one_fet11-01_12-58-54',
# ]
# excluded_session_keys = ['2006-6-08_14-26-15', '2006-6-09_1-22-43']
excluded_session_keys = []
excluded_session_contexts = [IdentifyingContext.try_init_from_session_key(session_str=v) for v in excluded_session_keys]
print(f'excluded_session_contexts: {excluded_session_contexts}')
included_session_contexts: List[IdentifyingContext] = [v for v in all_session_contexts if v not in excluded_session_contexts]
print(f'included_session_context: {included_session_contexts}')

# debug_print = True
debug_print = False
all_known_session_contexts: List[IdentifyingContext] = UserAnnotationsManager.get_all_known_sessions()
known_global_data_root_parent_paths = [Path('/Users/pho/data'), Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'W:/Data'), Path(r'/home/halechr/FastData'), Path(r'/Volumes/MoverNew/data')] # Path(r'/home/halechr/cloud/turbo/Data'), , Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'/home/halechr/turbo/Data'), 
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
print(f'global_data_root_parent_path: {global_data_root_parent_path}')
all_known_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, all_known_session_contexts)
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)
session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}

Automatic pdb calling has been turned OFF


/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/hdf5storage/utilities.py:44: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


scripts_output_path: /nfs/turbo/umms-kdiba/Pho/Output/gen_scripts
collected_outputs_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_outputs
collected_figures_path: /nfs/turbo/umms-kdiba/Pho/Output/collected_figures
excluded_session_contexts: []
included_session_context: [IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-08_14-26-15')>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-09_1-22-43')>, IdentifyingContext<('kdiba', 'gor01', 'one', '2006-6-12_15-55-31')>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19')>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-12_16-53-46')>, IdentifyingContext<('kdiba', 'vvp01', 'one', '2006-4-09_17-29-30')>, IdentifyingContext<('kdiba', 'vvp01', 'one', '2006-4-10_12-25-50')>, IdentifyingContext<('kdiba', 'vvp01', 'two', '2006-4-09_16-40-54')>, IdentifyingContext<('kdiba', 'vvp01', 'two', '2006-4-10_12-58-3')>, IdentifyingContext<('kdiba', 'pin01', 'one', '11-03_12-3-25')>, IdentifyingContext<('kdiba', 'pin01', 'one',

In [2]:

## Different run configurations:
active_phase_dict = {}
# ==================================================================================================================== #
# Custom Suffix and overrides                                                                                          #
# ==================================================================================================================== #
# '_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2]' ___________________________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = '_withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0'
# included_qclu_values = [1,2]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0" ## ends up being something like # job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0_tbin_25ms"

# # # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2, 4, 6, 7, 9]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0" ##
# included_qclu_values = [1, 2, 4, 6, 7, 9]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_124679-frateThresh_5.0"


# # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0" 
# included_qclu_values = [1, 2]
# minimum_inclusion_fr_Hz = 5.0
# job_suffix = f"_withNormalComputedReplays-qclu_12-frateThresh_5.0"

# # # "_withNormalComputedReplays-frateThresh_5.0-qclu_[1, 2, 4, 6, 7, 8, 9]" _______________________________________________ #
# active_phase_dict['override_custom_pickle_suffix'] = "_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 8, 9]-frateThresh_5.0"
included_qclu_values = [1, 2, 4, 6, 7, 8, 9]
minimum_inclusion_fr_Hz = 5.0
job_suffix = f"_withNormalComputedReplays-qclu_1246789-frateThresh_5.0"


# ==================================================================================================================== #
# Common:                                                                                                              #
# ==================================================================================================================== #

# laps_decoding_time_bin_size = None
laps_decoding_time_bin_size = 0.025
# laps_decoding_time_bin_size = 0.025

# ripple_decoding_time_bin_size = 0.058
# job_suffix='tbin_58ms'
ripple_decoding_time_bin_size = 0.025
# job_suffix='tbin_50ms'
# ripple_decoding_time_bin_size = 0.025
# job_suffix='tbin_25ms'


# job_suffix = f"{active_phase_dict['override_custom_pickle_suffix']}_tbin_25ms"
job_suffix = f"{job_suffix}_tbin_25ms"
# job_suffix = f"{job_suffix}_tbin_50ms"



# END ________________________________________________________________________________________________________________ #
print(f'job_suffix: {job_suffix}')


job_suffix: _withNormalComputedReplays-qclu_1246789-frateThresh_5.0_tbin_25ms


# 🔷 Maintenence and File Management

## Move misplaced files to session folders

### 2025-06-10 - Works to copy the .h5 files in each session after batch compute/export into the `collected_outputs_path`

In [ ]:
from pyphocorehelpers.gui.Jupyter.IPythonDryrunExecutionWidget import DryRunExecutionWidget
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_most_recent_files
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers

## INPUTS: good_session_concrete_folders, session_basedirs_dict, computation_script_paths
## INPUTS: BATCH_DATE_TO_USE
print(F'BATCH_DATE_TO_USE: "{BATCH_DATE_TO_USE}"')
# BATCH_DATE_TO_USE='2025-04-18'

print(f'collected_outputs_path: "{collected_outputs_path.as_posix()}"')
# session_basedirs_dict: Dict[IdentifyingContext, Path] = {a_session_folder.context:a_session_folder.path for a_session_folder in good_session_concrete_folders}
is_dryrun: bool = True
# is_dryrun: bool = False
debug_print: bool = False

# cuttoff_date = datetime(2025, 4, 17)
# cuttoff_date = datetime(2025, 6, 16)
cuttoff_date = datetime(2025, 6, 16)
# cuttoff_date = None


def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
    """ Builds output HDF5 file name
    Captures `BATCH_DATE_TO_USE`
    """
    # a_session_folder.context
    if session_context is not None:
        session_descr = session_context.session_name # '2006-6-07_16-40-19'
    _filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
    if len(args) > 0:
        _filename_list.extend([str(a_part) for a_part in args if a_part is not None])
    return separator_char.join(_filename_list)


_copy_exported_files_from_session_folder_to_collected_outputs_kwargs = dict(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, 
        cuttoff_date=cuttoff_date, 
        target_dir=collected_outputs_path, 
        custom_file_globs_dict={'h5': '*.h5'}, 
        rename_backup_basename_fn=_across_session_h5_output_basename_fn,
)



# Define the operation as a function that takes is_dryrun parameter
def copy_session_files(is_dryrun=True):
    """
    Captures: BATCH_DATE_TO_USE, cuttoff_date, collected_outputs_path, is_dryrun
    """
    from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers
    
    print(F'BATCH_DATE_TO_USE: "{BATCH_DATE_TO_USE}"')
    print(f'collected_outputs_path: "{collected_outputs_path.as_posix()}"')
    
    # Use the provided is_dryrun parameter
    copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(
        BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, 
        cuttoff_date=cuttoff_date, 
        target_dir=collected_outputs_path, 
        custom_file_globs_dict={'h5': '*.h5'}, 
        is_dry_run=is_dryrun, rename_backup_basename_fn=_across_session_h5_output_basename_fn,
    )
    
    return (copy_dict, moved_files_dict_files)

# Create the widget with your operation function
# widget = DryRunExecutionWidget(copy_session_files)


## OUTPUTS: _copy_exported_files_from_session_folder_to_collected_outputs_kwargs

In [ ]:


# copy_dict, moved_dict, (all_found_pipeline_pkl_files_dict, all_found_global_pkl_files_dict, all_found_pipeline_h5_files_dict) = try_move_pickle_files_on_GL(good_session_concrete_folders, session_basedirs_dict, computation_script_paths,
# 			is_dryrun=is_dryrun, debug_print=debug_print)

## Specify which files to match:    
# copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, cuttoff_date=cuttoff_date, target_dir=collected_outputs_path, custom_file_globs_dict={
#     # 'pkl': '*standalone_wcorr_ripple_shuffle_data_only*.pkl',
#     # 'csv': '*ripple_WCorrShuffle_df*.csv',
#     'h5': '*.h5',
# }, is_dry_run=is_dryrun)

copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(**(_copy_exported_files_from_session_folder_to_collected_outputs_kwargs | dict(custom_file_globs_dict={
    # 'pkl': '*standalone_wcorr_ripple_shuffle_data_only*.pkl',
    # 'csv': '*ripple_WCorrShuffle_df*.csv',
    'h5': '*.h5',
})), is_dry_run=is_dryrun)

copy_dict

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionHelpers

def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
	""" Builds output HDF5 file name
	Captures `BATCH_DATE_TO_USE`
	"""
	# a_session_folder.context
	if session_context is not None:
		session_descr = session_context.session_name # '2006-6-07_16-40-19'
	_filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
	if len(args) > 0:
		_filename_list.extend([str(a_part) for a_part in args if a_part is not None])
	return separator_char.join(_filename_list)


is_dryrun: bool = True
# is_dryrun: bool = False

assert collected_outputs_path.exists()
copy_dict, moved_files_dict_files = AcrossSessionHelpers._copy_exported_files_from_session_folder_to_collected_outputs(BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, cuttoff_date=cuttoff_date, target_dir=collected_outputs_path, is_dry_run=is_dryrun, rename_backup_basename_fn=_across_session_h5_output_basename_fn)
copy_dict


# `gen_scripts` batch figure export copy to `collected_figures` folder via `copy_batch_output_figures_to_common_figures_dir(...)`

In [3]:
import shutil
from pathlib import Path
from typing import List, Optional
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_batch_output_figures_to_common_figures_dir

## Move figures
# generate_figures_script_paths = [Path(v) for v in ['C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_one_2006-6-07_11-26-53/figures_kdiba_gor01_one_2006-6-07_11-26-53.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/figures_kdiba_gor01_one_2006-6-08_14-26-15.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/figures_kdiba_gor01_one_2006-6-09_1-22-43.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31/figures_kdiba_gor01_one_2006-6-12_15-55-31.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/figures_kdiba_gor01_two_2006-6-07_16-40-19.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/figures_kdiba_gor01_two_2006-6-08_21-16-25.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/figures_kdiba_gor01_two_2006-6-09_22-24-40.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/figures_kdiba_gor01_two_2006-6-12_16-53-46.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/figures_kdiba_vvp01_one_2006-4-09_17-29-30.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/figures_kdiba_vvp01_one_2006-4-10_12-25-50.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/figures_kdiba_vvp01_two_2006-4-09_16-40-54.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/figures_kdiba_vvp01_two_2006-4-10_12-58-3.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/figures_kdiba_pin01_one_11-02_17-46-44.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/figures_kdiba_pin01_one_11-02_19-28-0.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/figures_kdiba_pin01_one_11-03_12-3-25.py',
#  'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/figures_kdiba_pin01_one_fet11-01_12-58-54.py']]

# from pathlib import Path as PosixPath
# script_folders = [PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-07_11-26-53'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25'), PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54')]

generate_figures_script_paths = ['/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-07_11-26-53/figures_kdiba_gor01_one_2006-6-07_11-26-53.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-08_14-26-15/figures_kdiba_gor01_one_2006-6-08_14-26-15.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-09_1-22-43/figures_kdiba_gor01_one_2006-6-09_1-22-43.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_one_2006-6-12_15-55-31/figures_kdiba_gor01_one_2006-6-12_15-55-31.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-07_16-40-19/figures_kdiba_gor01_two_2006-6-07_16-40-19.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-08_21-16-25/figures_kdiba_gor01_two_2006-6-08_21-16-25.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-09_22-24-40/figures_kdiba_gor01_two_2006-6-09_22-24-40.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_gor01_two_2006-6-12_16-53-46/figures_kdiba_gor01_two_2006-6-12_16-53-46.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-09_17-29-30/figures_kdiba_vvp01_one_2006-4-09_17-29-30.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_one_2006-4-10_12-25-50/figures_kdiba_vvp01_one_2006-4-10_12-25-50.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-09_16-40-54/figures_kdiba_vvp01_two_2006-4-09_16-40-54.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_vvp01_two_2006-4-10_12-58-3/figures_kdiba_vvp01_two_2006-4-10_12-58-3.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-02_17-46-44/figures_kdiba_pin01_one_11-02_17-46-44.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-02_19-28-0/figures_kdiba_pin01_one_11-02_19-28-0.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_11-03_12-3-25/figures_kdiba_pin01_one_11-03_12-3-25.py', '/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts/run_kdiba_pin01_one_fet11-01_12-58-54/figures_kdiba_pin01_one_fet11-01_12-58-54.py']
generate_figures_script_paths = [Path(v) for v in generate_figures_script_paths]

_copied_outputs = copy_batch_output_figures_to_common_figures_dir(generate_figures_script_paths=generate_figures_script_paths, common_destination=collected_figures_path, previous_gen_scripts_path=Path('/nfs/turbo/umms-kdiba/Pho/Output/gen_scripts'), curr_gen_scripts_path=None)
_copied_outputs

[PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/maze1_any_any_1d_placefields_kdiba_gor01_one_2006-6-07_11-26-53.png'),
 PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/kdiba_gor01_one_2006-6-07_11-26-53_maze2_even_even__display_1d_placefield_validations_kdiba_gor01_one_2006-6-07_11-26-53.pdf'),
 PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/maze_even_even_1d_placefields_kdiba_gor01_one_2006-6-07_11-26-53.png'),
 PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/kdiba_gor01_one_2006-6-07_11-26-53_maze1_odd_odd__display_1d_placefield_validations_kdiba_gor01_one_2006-6-07_11-26-53.pdf'),
 PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/maze_odd_odd_1d_placefields_kdiba_gor01_one_2006-6-07_11-26-53.png'),
 PosixPath('/nfs/turbo/umms-kdiba/Pho/Output/collected_figures/kdiba_gor01_one_2006-6-07_11-26-53_maze2_any_any__display_1d_placefield_validations_kdiba_gor01_one_2006-6-07_11-26-53.pdf'),
 PosixPath('/nfs/turbo/umms-k

## ⛓️🟢 Post-Processing of batch outputs:

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import save_filelist_to_text_file


#TODO 2025-04-18 07:07: - [ ] Also move .h5 files with parameter-included filenames, like 'pipeline_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0.h5'
# Example:    "W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\output\pipeline_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 9]-frateThresh_5.0.h5

debug_print = True
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

# included_h5_paths = [Path(get_file_str_if_file_exists(v.pipeline_results_h5)).resolve() for v in good_session_concrete_folders] # 2025-04-18 07:25 - Old style 'pipeline_.h5'
included_h5_paths = []
for v in good_session_concrete_folders:
    ## find all relevant 'pipeline_*.h5' files
    matching_files = list(v.output_folder.glob('pipeline_*.h5'))
    included_h5_paths.extend([p.resolve() for p in matching_files])
	
check_output_h5_files(included_h5_paths)
included_h5_paths
## OUTPUTS: included_h5_paths

# def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
# 	""" Captures `BATCH_DATE_TO_USE` """
# 	# a_session_folder.context
# 	if session_context is not None:
# 		session_descr = session_context.session_name # '2006-6-07_16-40-19'
# 	_filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
# 	if len(args) > 0:
# 		_filename_list.extend([str(a_part) for a_part in args if a_part is not None])
# 	return separator_char.join(_filename_list)


def _across_session_h5_output_basename_fn(session_context: Optional[IdentifyingContext], session_descr: Optional[str], basename: str, *args, separator_char: str = "_"):
	""" Builds output HDF5 file name
	Captures `BATCH_DATE_TO_USE`
	"""
	# a_session_folder.context
	if session_context is not None:
		session_descr = session_context.session_name # '2006-6-07_16-40-19'
	_filename_list = [BATCH_DATE_TO_USE, session_descr, basename]
	if len(args) > 0:
		_filename_list.extend([str(a_part) for a_part in args if a_part is not None])
	return separator_char.join(_filename_list)


# copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn, only_include_file_types=['h5']) # , rename_backup_suffix=BATCH_DATE_TO_USE
# copy_h5_dict

from pyphoplacecellanalysis.General.Batch.runBatch import get_file_path_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_folder_files_to_target_dir

custom_file_types_dict = {
	                    # 'custom_h5': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'pipeline_withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0.h5').resolve())),
                        #   'PHONEW.evt': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'{a_session_folder.context.session_name}.PHONEW.evt').resolve())),
                        }

copy_h5_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, target_dir=collected_outputs_path, backup_mode=BackupMethods.CommonTargetDirectory, rename_backup_basename_fn=_across_session_h5_output_basename_fn,
#  only_include_file_types=['h5'],
    only_include_file_types=[], # exclude the un-parameterized pipeline_results.h5
    custom_file_types_dict=custom_file_types_dict) # , rename_backup_suffix=BATCH_DATE_TO_USE
copy_h5_dict


## INPUTS: target_dir, BATCH_DATE_TO_USE
h5_filelist_output_filename=f'{BATCH_DATE_TO_USE}_all_sessions_h5_filelist.txt'
h5_filelist_output_file_path = Path(target_dir).joinpath(h5_filelist_output_filename).resolve() # Use Default
print(f'h5_filelist_output_file_path: {h5_filelist_output_file_path}')
_out_string, filelist_path = save_filelist_to_text_file(included_h5_paths, filelist_path=h5_filelist_output_file_path, debug_print=True) # r"W:\Data\all_sessions_h5_filelist_2024-03-28_Apogee.txt"
filelist_path

## OUTPUTS: copy_h5_dict, 

In [ ]:
copy_h5_dict

# Output File Processing Helpers

## Copy exported `pipeline_results.h5` files into the `collected_outputs` folder, adding the current date
```
copying "W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\output\pipeline_results.h5"
		 -> "K:\scratch\collected_outputs\2024-06-12_Apogee_2006-6-08_14-26-15_pipeline_results.h5"...
```

In [ ]:
## INPUTS: copy_h5_dict

a_copy_dict = copy_h5_dict

# Copies each file sequentially to the collected_outputs directory, and then builds an output file list
## INPUT a_copy_dict
moved_files_dict_h5_files = copy_movedict(a_copy_dict)
moved_files_dict_h5_files
# INPUTS: active_filelist_prefix, target_dir
# active_filelist_prefix: str = 'backed_up_files'
active_filelist_prefix: str = 'session_h5_files'

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path

moved_files_copydict_output_filename=f'{active_filelist_prefix}_copydict_{BATCH_DATE_TO_USE}.csv'
moved_files_copydict_file_path = target_dir.joinpath(moved_files_copydict_output_filename).resolve() # Use Default
print(f'moved_files_copydict_file_path: "{moved_files_copydict_file_path}"')

_out_string, filedict_out_path = save_copydict_to_text_file(moved_files_dict_h5_files, moved_files_copydict_file_path, debug_print=True)

## 2024-09-04 - Batch Output Files

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_inst_fr_data_to_across_session_pkl

## Complete Concise Run:
# RESULT_DATE_TO_USE = '2024-09-26'
# RESULT_DATE_TO_USE = '2025-04-17'
# RESULT_DATE_TO_USE = '2025-04-22'
# RESULT_DATE_TO_USE = '2025-06-11'
RESULT_DATE_TO_USE = '2025-06-17'

RESULT_DATE_TO_USE: str = BATCH_DAY_DATE
print(f'RESULT_DATE_TO_USE: {RESULT_DATE_TO_USE}')

# instantaneous_time_bin_size_seconds_list = [0.002, 0.005, 0.025, 1000.0]
instantaneous_time_bin_size_seconds_list = [1000.0]

across_sessions_recomputed_instantaneous_fr_dict, moved_files_dict_files, (filelist_path, filedict_out_path) = copy_session_inst_fr_data_to_across_session_pkl(RESULT_DATE_TO_USE=RESULT_DATE_TO_USE, collected_outputs_path=collected_outputs_path, instantaneous_time_bin_size_seconds_list=instantaneous_time_bin_size_seconds_list)


# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_recomputed_instantaneous_fr_dict

# ! After Sessions are processed and their `.h5` files are exported, build the combined tables
The combined (across session) tables are saved out into .csv and .pkl formats: `W:\Data\2024-06-12_Apogee\neuron_identities_table.csv` -- ideally this would be in the combined outputs folder

In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import read_filelist_from_text_file
    

# filelist_path = Path(r"K:/scratch/collected_outputs/2025-06-11_GL_all_sessions_h5_filelist.txt").resolve() # APOGEE
filelist_path = Path('/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2025-06-17_GL_all_sessions_h5_filelist.txt').resolve() # GL

read_hdf5_output_paths = read_filelist_from_text_file(filelist_path=filelist_path, debug_print=False)
read_hdf5_output_paths


In [ ]:
read_hdf5_output_paths

### 2025-06-11 - Get the .h5 files that were previously output (or transferred) to the CollectedOutputs directory

In [ ]:
## INPUTS: cuttoff_date

included_h5_paths = []
## find all relevant 'pipeline_*.h5' files
matching_files = list(collected_outputs_path.glob('*_pipeline_results.h5'))
included_h5_paths = deepcopy(matching_files)
# included_h5_paths

# ## Doesn't work at all
# find_most_recent_files_kwargs = dict(should_fallback_to_filesystem_modification_datetime=True, should_print_unparsable_filenames=False, debug_print=debug_print)
# _, a_most_recent_parsed_files_df, all_parsed_files_df  = find_most_recent_files(found_session_export_paths=matching_files, cuttoff_date=cuttoff_date, **find_most_recent_files_kwargs)
# a_most_recent_parsed_files_df['filename'] = a_most_recent_parsed_files_df['path'].map(lambda x: x.name)
# a_most_recent_parsed_files_df

# all_parsed_files_df
check_output_h5_files(included_h5_paths)
included_h5_paths




In [ ]:
[v.as_posix() for v in included_h5_paths]

In [ ]:
## Compare the path's filename with the `job_suffix` to see if it's a match. Filenames might contain extra spaces, commas (',') or hard-brackets ('[]') that should be ignored for comparison's sake:
# a_path: Path = Path('/nfs/turbo/umms-kdiba/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 8, 9]-frateThresh_5.0.h5')
# 'pipeline_withNormalComputedReplays-qclu_[1, 2, 4, 6, 7, 8, 9]-frateThresh_5.0.h5'
# Normalize the strings by removing spaces, commas and brackets
def normalize_str(s: str) -> str:
    return s.replace(' ', '').replace(',', '').replace('[', '').replace(']', '')

# display(included_h5_paths)
# filename_normalized_job_suffix: str = normalize_str(job_suffix)
# filename_normalized_job_suffix.split('-'

# filename_normalized_job_suffix: str = '_withNormalComputedReplays-qclu_1246789-frateThresh_5.0' # #TODO 2025-06-11 13:55: - [ ] HARDCODED
filename_normalized_job_suffix: str = '_withNormalComputedReplays-qclu_12-frateThresh_5.0' # #TODO 2025-06-11 13:55: - [ ] HARDCODED

display(filename_normalized_job_suffix) # '_withNormalComputedReplays-qclu_1246789-frateThresh_5.0_tbin_25ms'
# [normalize_str(a_path.name) for a_path in included_h5_paths] 

# '_withNormalComputedReplays-qclu_1246789-frateThresh_5.0_tbin_25ms' in 'pipeline_withNormalComputedReplays-qclu_1246789-frateThresh_5.0.h5'

# 'pipeline_withNormalComputedReplays-qclu_1246789-frateThresh_5.0.h5',
#  'pipeline_withNormalComputedReplays-qclu_12-frateThresh_5.0.h5',
#  'pipeline_withNormalComputedReplays-frateThresh_5.0-qclu_124679.h5',
#  'pipeline_results_2024-09-11_GL.h5',
#  'pipeline_withNormalComputedReplays-qclu_124679-frateThresh_4.0.h5',
#  'pipeline_results_2023-10-25_BidirectionalOnly.h5',
#  'pipeline_results.h5',
#  'pipeline_results_Pre2024-07-16.h5',
#  'pipeline_withNormalComputedReplays-frateThresh_5.0-qclu_12.h5',
#  'pipeline_results_2024-09-26_GL.h5',
#  'pipeline_withNormalComputedReplays-qclu_12-frateThresh_4.0.h5',
# Check if normalized suffix is in normalized filename
active_included_h5_paths = [a_path for a_path in included_h5_paths if (filename_normalized_job_suffix in normalize_str(a_path.stem))]
print(f'including: {len(active_included_h5_paths)}/{len(included_h5_paths)} h5 paths for job_suffix: {job_suffix}')
active_included_h5_paths


In [ ]:
## Use all `included_h5_paths`
active_included_h5_paths = deepcopy(included_h5_paths)


In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## FILE_OUTPUTS: '/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/2024-10-22_GL'

## INPUTS: included_session_contexts, active_included_h5_paths, global_data_root_parent_path, BATCH_DATE_TO_USE, job_suffix
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, active_included_h5_paths, should_restore_native_column_types=True, )



# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_path
assert target_dir.exists(), f'target_dir: "{target_dir}" does not exist!'
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, active_included_h5_paths, override_output_parent_path=target_dir, output_path_suffix=f'{BATCH_DATE_TO_USE}')
curr_collected_outputs_folder = Path(output_path_dicts['neuron_replay_stats_table']['.csv']).resolve().parent
filesystem_path_folder_contents_widget(curr_collected_outputs_folder)

In [ ]:
neuron_replay_stats_table

# 2024-06-12 11:18: - [ ] HACK: reads back in the exported `neuron_replay_stats_table` CSV to fix an issue with pd.NA and np.nan not reading a NaNs and the dtypes of the columns being messed up on load:

In [ ]:
_curr_read_path = output_path_dicts['neuron_replay_stats_table']['.csv']
# _curr_read_path = Path('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee/neuron_replay_stats_table.csv').resolve()
neuron_replay_stats_table: pd.DataFrame = pd.read_csv(_curr_read_path, na_values=['', 'nan', 'np.nan','<NA>'],
										dtype={
											'long_pf_peak_x': 'float64', 
											'short_pf_peak_x': 'float64',
											'long_LR_pf2D_peak_x': 'float64',
											'long_LR_pf2D_peak_y': 'float64',
											'long_RL_pf2D_peak_x': 'float64',
											'long_RL_pf2D_peak_y': 'float64',
											'short_LR_pf2D_peak_x': 'float64',
											'short_LR_pf2D_peak_y': 'float64',
											'short_RL_pf2D_peak_x': 'float64',
											'short_RL_pf2D_peak_y': 'float64',
											'long_LR_pf1D_peak': 'float64',
											'long_RL_pf1D_peak': 'float64',
											'short_LR_pf1D_peak': 'float64',
											'short_RL_pf1D_peak': 'float64',
											'peak_diff_RL_pf1D_peak': 'float64',
											'peak_diff_LR_pf1D_peak': 'float64'
										}, index_col='neuron_uid')
neuron_replay_stats_table


In [ ]:
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import TruncationCheckingResults

# truncation_checking_aclus_dict, jonathan_firing_rate_analysis_result.neuron_replay_stats_df = truncation_checking_result.build_truncation_checking_aclus_dict(neuron_replay_stats_df=jonathan_firing_rate_analysis_result.neuron_replay_stats_df)

# 'is_rate_extrema', 'is_refined_exclusive', 'is_refined_LxC', 'is_refined_SxC'
# 'is_long_peak_left_cap', 'is_long_peak_right_cap', 'is_long_peak_either_cap'

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _add_cell_remapping_category

neuron_replay_stats_df = deepcopy(neuron_replay_stats_table)

neuron_replay_stats_df, (non_disappearing_endcap_cells_df, disappearing_endcap_cells_df, minorly_changed_endcap_cells_df, significant_distant_remapping_endcap_cells_df) = _add_cell_remapping_category(neuron_replay_stats_df=neuron_replay_stats_df,
													   loaded_track_limits = {'long_xlim': np.array([59.0774, 228.69]), 'short_xlim': np.array([94.0156, 193.757]), 'long_ylim': np.array([138.164, 146.12]), 'short_ylim': np.array([138.021, 146.263])},
)
# neuron_replay_stats_df
significant_distant_remapping_endcap_cells_df

In [ ]:
neuron_replay_stats_df

In [ ]:
# non_disappearing_endcap_cells_df
significant_distant_remapping_endcap_cells_df
# 'track_membership'

## Required for Interactive Plotting and figure displace

In [ ]:
neuron_replay_stats_table.index.unique() #['neuron_uid'].unique()

In [ ]:

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
num_sessions: int = 13
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Extract output files from all completed sessions:

# 2023-10-04 - Load Saved across-sessions-data, process, and produce figures

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'
# across_session_result_long_short_recomputed_inst_firing_rate_file = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
across_session_result_long_short_recomputed_inst_firing_rate_file = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2025-06-11_1000.0.pkl').resolve() # "K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2025-06-11_1000.0.pkl"
across_session_result_long_short_recomputed_inst_firing_rate_file.exists()


In [ ]:
# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## INPUTS: BATCH_DATE_TO_USE, BATCH_DATE_TO_USE
print(f"BATCH_DATE_TO_USE: {BATCH_DATE_TO_USE}")
## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

across_session_result_long_short_recomputed_inst_firing_rate_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# "/nfs/turbo/umms-kdiba/Pho/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-10-22_GL.pkl"

# AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path,
#                                                  inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)

AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_recomputed_instantaneous_fr_dict, global_data_root_parent_path=collected_outputs_path.resolve(),
												 inst_fr_output_filename=across_session_result_long_short_recomputed_inst_firing_rate_filename)


# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"

# '/nfs/turbo/umms-kdiba/Data/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-10_GL.pkl'
# Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
# output_path_suffix: str = '2024-09-11_GL'
# output_path_suffix: str = '2024-09-26_GL'
output_path_suffix: str = BATCH_DATE_TO_USE
# graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=global_data_root_parent_path, BATCH_DATE_TO_USE=BATCH_DATE_TO_USE, plotting_enabled=True)
graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix, plotting_enabled=True)


In [ ]:
from PendingNotebookCode import build_and_merge_all_sessions_joined_neruon_fri_df

all_sessions_joined_neruon_fri_df, out_path = build_and_merge_all_sessions_joined_neruon_fri_df(global_data_root_parent_path, BATCH_DATE_TO_USE, included_session_contexts=included_session_contexts)

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager, SessionCellExclusivityRecord
from neuropy.utils.result_context import IdentifyingContext

annotation_man = UserAnnotationsManager()

LxC_uids = []
SxC_uids = []

for a_ctxt in included_session_contexts:
	session_uid = a_ctxt.get_description(separator="|", include_property_names=False)
	session_uid
	session_cell_exclusivity: SessionCellExclusivityRecord = annotation_man.annotations[a_ctxt].get('session_cell_exclusivity', None)
	LxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.LxC])
	SxC_uids.extend([f"{session_uid}|{aclu}" for aclu in session_cell_exclusivity.SxC])
	
# [a_ctxt.get_description(separator="|", include_property_names=False) for a_ctxt in included_session_contexts]

long_short_fr_indicies_analysis_table['XxC_status'] = 'Shared'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, LxC_uids), 'XxC_status'] = 'LxC'
long_short_fr_indicies_analysis_table.loc[np.isin(long_short_fr_indicies_analysis_table.neuron_uid, SxC_uids), 'XxC_status'] = 'SxC'

long_short_fr_indicies_analysis_table
## 2023-10-11 - Get the long peak location

long_short_fr_indicies_analysis_table['long_pf_peak_x'] = neuron_replay_stats_table['long_pf_peak_x']
long_short_fr_indicies_analysis_table

## SAVE:
# long_short_fr_indicies_analysis_table_filename = 'output/2023-10-07_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table_filename: str = f'output/{BATCH_DATE_TO_USE}_long_short_fr_indicies_analysis_table.csv'
long_short_fr_indicies_analysis_table.to_csv(long_short_fr_indicies_analysis_table_filename)
print(f'saved: {long_short_fr_indicies_analysis_table_filename}')

# Loading

In [ ]:
## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-10-04-GL.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-04-GL-Recomp.pkl'
# inst_fr_output_filename='across_session_result_long_short_recomputed_inst_firing_rate_2023-10-07.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# inst_fr_output_load_filepath: Path = Path('/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl').resolve()
# inst_fr_output_load_filepath: Path = collected_outputs_path.joinpath('across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl').resolve()
inst_fr_output_load_filepath = deepcopy(across_session_result_long_short_recomputed_inst_firing_rate_file)
# "/nfs/turbo/umms-kdiba/Data/Output/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-26_GL.pkl"
assert inst_fr_output_load_filepath.exists()
inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
 
## Load all across-session tables from the pickles:
# output_path_suffix: str = f'2023-10-07'
# output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
# output_path_suffix: str = '2024-06-12_GL'
# output_path_suffix: str = '2024-09-03_GL'
output_path_suffix: str = '2025-06-11'
# neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_path, output_path_suffix=output_path_suffix)
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')

### 2023-11-15 - For manual testing

In [ ]:
# ## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=global_data_root_parent_path, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [ ]:
across_sessions_instantaneous_frs_list

# KDiba Session Discovery - Determines contexts to process - 2024-09-19

In [ ]:
# from pathlib import Path
# from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
# from neuropy.utils.result_context import print_identifying_context_array_code

# known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
# global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
# assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
sessions_df, export_folder_path = KDibaOldDataSessionFormatRegisteredClass.find_build_and_save_sessions_experiment_datetime_df_csv(
																			# global_data_root_parent_path=global_data_root_parent_path,
																			export_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/sessions_experiment_datetime_df.csv').resolve(),
																			bad_sessions_csv_path=Path('EXTERNAL/PhoDibaPaper2024Book/data/2024-09-23_bad_sessions_table.csv').resolve(),
)
sessions_df = sessions_df.reset_index(drop=True, inplace=False)
sessions_df


In [ ]:
# old_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), 
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), #
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     # IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), #
# ]
	
# new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
# ]

# completely_new_included_session_contexts = [
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_14-28-51'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_16-48-29'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_13-28-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_19-11-57'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-26_13-22-13'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_3-23-37'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_17-33-28'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_17-6-14'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_11-0-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_11-43-50'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-27_18-21-57'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-21_10-24-35'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_17-53-55'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_16-37-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-11_15-16-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_16-48-9'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-2-0'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_13-55-7'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_14-59-23'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-21_11-19-2'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_21-2-40'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-12_14-39-31'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-25_17-17-6'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_15-46-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-18_15-38-2'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-28_12-17-27'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-16_15-12-23'),
#     IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-07_11-26-53'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-19_12-35-59'),
#     IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
#     IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-03_20-28-3'),
# ]


In [ ]:
completely_new_included_session_contexts = set(new_included_session_contexts) - set(old_included_session_contexts)
print_identifying_context_array_code(completely_new_included_session_contexts, array_name='completely_new_included_session_contexts')

# Old File Cleanup

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import OldFileArchiver

""" 
20240911133516-loadedSessPickle.pkl
20240911131815-loadedSessPickle.pkl
20240911131513-loadedSessPickle.pkl
"""

# deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders, is_dryrun=True)
always_delete_patterns = {
	'.pkltmp': '*.pkltmp',
	'.pkl.bak': '*.pkl.bak',
}
# always_delete_patterns = {} # disable non-conditional deletes while the pipeline is running in batch

# Define patterns to conditionally delete
conditional_delete_patterns = {
	# '.pkl': '*.pkl',
	'pipeline.pkl': '*loadedSessPickle*.pkl',
	'global.pkl': '*global_computation_results*.pkl',
	## Numpy files:
	'position.npy': '*.position.npy',
	'mua.npy': '*.mua.npy',
	'pbe.npy': '*.pbe.npy',
	'non_pbe.npy': '*.non_pbe.npy',
	'interpolated_spike_positions.npy': '*.interpolated_spike_positions.npy',
	'non_pbe.npy': '*.non_pbe.npy',
}

## Numpy files:
numpy_files_patterns = {
	'position.npy': '*.position.npy',
	'mua.npy': '*.mua.npy',
	'pbe.npy': '*.pbe.npy',
	'non_pbe.npy': '*.non_pbe.npy',
	'interpolated_spike_positions.npy': '*.interpolated_spike_positions.npy',
	'non_pbe.npy': '*.non_pbe.npy',
	'ripple.npy': '*.ripple.npy'
}


mat_files_patterns = {
    'epochs_info.mat':'*.epochs_info.mat',
    'laps_info.mat': '*.laps_info.mat',
    'position_info.mat': '*.position_info.mat',
    'spikes.mat': '*.spikes.mat',
}


always_delete_patterns = always_delete_patterns | numpy_files_patterns

# Define the cutoff date
# conditional_delete_files_cutoff_date = datetime(2023, 9, 1)  # Example cutoff date
conditional_delete_files_cutoff_date = datetime(2024, 9, 1)
# conditional_delete_files_cutoff_date = datetime(2025, 2, 20)
# conditional_delete_files_cutoff_date = datetime(2025, 3, 20)
# conditional_delete_files_cutoff_date = None

# deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders, conditional_delete_files_cutoff_date=conditional_delete_files_cutoff_date, is_dryrun=True)
deleted_file_list = OldFileArchiver.remove_backup_files_from_session_data_folders(good_session_concrete_folders=all_known_session_concrete_folders,
																				  always_delete_patterns=always_delete_patterns,
																				  conditional_delete_patterns=conditional_delete_patterns,
																				   conditional_delete_files_cutoff_date=conditional_delete_files_cutoff_date,
																				#    is_dryrun=True,
																				   is_dryrun=False,
																				   )
deleted_file_list

In [ ]:
# '2025-04-05_0500PM-kdiba_gor01_two_2006-6-12_16-53-46_normal_computed_[1, 2, 4, 6, 7, 8, 9]_5.0-(FAT)_tbin-0.025.csv':'2025-04-05_0500PM-kdiba_gor01_two_2006-6-12_16-53-46_normal_computed_[1, 2, 4, 6, 7, 8, 9]_5.0-(FAT)_tbin-0.025.csv'

# Correctly parsed filename: "2025-03-04_0410AM-kdiba_pin01_one_11-03_12-3-25__withNormalComputedReplays-qclu_[1, 2]-frateThresh_5.0-(ripple_all_scores_merged_df)_tbin-0.025.csv"

rename_parts_dict = {'normal_computed':'_withNormalComputedReplays',
}


In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import OldFileArchiver

## INPUTS: collected_outputs_path, excluded_or_outdated_files_list
archive_folder = OldFileArchiver.archive_old_files(collected_outputs_path=collected_outputs_path, excluded_or_outdated_files_list=excluded_or_outdated_files_list, is_dry_run=True)

archive_folder